In [1]:
import numpy as np
import pandas as pd
import random
import math
from sklearn import metrics

In [2]:
# 预载数据
raw_rating_data = pd.read_csv('test2.csv')
all_users_id = list(set(list(raw_rating_data['userId'])))
all_users_id.sort()
last_user_id = max(all_users_id)
all_movies_id = set(list(raw_rating_data['movieId']))
trans = list(all_movies_id)
trans.sort()                                             # 电影编号从小到大排序

user_movie_df = pd.read_csv('user_rating_matrix.csv')

In [3]:
# 生成随机数种子并保存
train_movie_num = int(len(all_movies_id)*0.8)
train_user_num = int(len(all_users_id)*0.8)
train_movie_index = random.sample(range(1,len(all_movies_id)+1),train_movie_num)  # 在所有电影中随机取80%为训练集
test_movie_index = list(set(list(range(1, len(all_movies_id)+1))) ^ set(train_movie_index))
train_user_index = random.sample(range(0,len(all_users_id)),train_user_num)  # 在所有用户中随机取80%为训练集
test_user_index = list(set(list(range(0, len(all_users_id)))) ^ set(train_user_index))
# 基于用户的需要先观测到相似用户对待预测项目的评分，因此需要多出item，但是user数目不变
# 基于物品的需要先观测到相似物品被指定用户的评分，因此需要多出user，但是item数目不变

zuoxiajiao_df = user_movie_df.loc[train_user_index]
zuoxiajiao_df = zuoxiajiao_df[[str(i) for i in train_movie_index]]
zuoxiajiao_df.to_csv('zuoxiajiao_df.csv') # 数据转化为CSV，并且将index行隐藏，得到每个用户的浏览新闻类别统计矩阵（28501×38）
print("输出完成1")

KeyboardInterrupt: 

In [15]:
youshangjiao_df = user_movie_df.loc[test_user_index]
youshangjiao_df = youshangjiao_df[[str(i) for i in test_movie_index]]
youshangjiao_df.to_csv('youshangjiao_df.csv') # 数据转化为CSV，并且将index行隐藏，得到每个用户的浏览新闻类别统计矩阵（28501×38）
print("输出完成2")
zuoshangjiao_df = user_movie_df.loc[test_user_index]
zuoshangjiao_df = zuoshangjiao_df[[str(i) for i in train_movie_index]]
zuoshangjiao_df.to_csv('zuoshangjiao_df.csv') # 数据转化为CSV，并且将index行隐藏，得到每个用户的浏览新闻类别统计矩阵（28501×38）
print("输出完成3")
youxiajiao_df = user_movie_df.loc[train_user_index]
youxiajiao_df = youxiajiao_df[[str(i) for i in test_movie_index]]
youxiajiao_df.to_csv('youxiajiao_df.csv') # 数据转化为CSV，并且将index行隐藏，得到每个用户的浏览新闻类别统计矩阵（28501×38）
print("输出完成4")

输出完成2
输出完成3
输出完成4


In [45]:
def NBC_UBCF(condition, observe):
    index5=[i for i,x in enumerate(condition) if x==5]
    index4=[i for i,x in enumerate(condition) if x==4]
    index3=[i for i,x in enumerate(condition) if x==3]
    index2=[i for i,x in enumerate(condition) if x==2]
    index1=[i for i,x in enumerate(condition) if x==1]
    index0=[i for i,x in enumerate(condition) if x==0]
    all_len = len(condition)-len(index0)
    index_all = [index0,index1,index2,index3,index4,index5]
    def cal_condp(patial_feature, ob):
        if ob==0:
            return [1,1,1,1,1]                                    # 如果ob到的结果是这个用户没有评测，则返回五个均1值
        patial_feature1= patial_feature[index1]
        patial_feature2= patial_feature[index2]
        patial_feature3= patial_feature[index3]
        patial_feature4= patial_feature[index4]
        patial_feature5= patial_feature[index5]
        num=0
        zero_num=0
        for i in patial_feature1:
            if i==ob:
                num=num+1
            if i==0:
                zero_num = zero_num+1
        condp1=(num+0.01)/(len(index1)-zero_num+0.05)
        num=0
        zero_num=0
        for i in patial_feature2:
            if i==ob:
                num=num+1
            if i==0:
                zero_num = zero_num+1
        condp2=(num+0.01)/(len(index2)-zero_num+0.05)
        num=0
        zero_num=0
        for i in patial_feature3:
            if i==ob:
                num=num+1
            if i==0:
                zero_num = zero_num+1
        condp3=(num+0.01)/(len(index3)-zero_num+0.05)
        num=0
        zero_num=0
        for i in patial_feature4:
            if i==ob:
                num=num+1
            if i==0:
                zero_num = zero_num+1
        condp4=(num+0.01)/(len(index4)-zero_num+0.05)
        num=0
        zero_num=0
        for i in patial_feature5:
            if i==ob:
                num=num+1
            if i==0:
                zero_num = zero_num+1
        condp5=(num+0.01)/(len(index5)-zero_num+0.05)
        return [condp1,condp2,condp3,condp4,condp5]
    condp_all = np.array([cal_condp(train_df[i],observe[i]) for i in range(len(observe))])
    condp1 = sum([math.log(i, 2) for i in condp_all[:,0]])+math.log((len(index1)+0.01)/(all_len+0.05), 2)
    condp2 = sum([math.log(i, 2) for i in condp_all[:,1]])+math.log((len(index2)+0.01)/(all_len+0.05), 2)
    condp3 = sum([math.log(i, 2) for i in condp_all[:,2]])+math.log((len(index3)+0.01)/(all_len+0.05), 2)
    condp4 = sum([math.log(i, 2) for i in condp_all[:,3]])+math.log((len(index4)+0.01)/(all_len+0.05), 2)
    condp5 = sum([math.log(i, 2) for i in condp_all[:,4]])+math.log((len(index5)+0.01)/(all_len+0.05), 2)
    all_condp = np.array([condp1,condp2,condp3,condp4,condp5])
    max_index = np.argmax(all_condp)
    all_condp = all_condp-all_condp[max_index]
    reli = 1/sum([2**i for i in all_condp])
    reli2 = (2**all_condp[3]+2**all_condp[4])/sum([2**i for i in all_condp])
    #ondp1 = np.prod(condp_all[:,0])*(len(index1)+0.01)/(all_len+0.05)
    #ondp2 = np.prod(condp_all[:,1])*(len(index2)+0.01)/(all_len+0.05)
    #ondp3 = np.prod(condp_all[:,2])*(len(index3)+0.01)/(all_len+0.05)
    #ondp4 = np.prod(condp_all[:,3])*(len(index4)+0.01)/(all_len+0.05)
    #ondp5 = np.prod(condp_all[:,4])*(len(index5)+0.01)/(all_len+0.05)
    return [max_index+1, reli, reli2], [condp1,condp2,condp3,condp4,condp5]

In [46]:
def NBC_IBCF(condition, observe):
    index5=[i for i,x in enumerate(condition) if x==5]
    index4=[i for i,x in enumerate(condition) if x==4]
    index3=[i for i,x in enumerate(condition) if x==3]
    index2=[i for i,x in enumerate(condition) if x==2]
    index1=[i for i,x in enumerate(condition) if x==1]
    index0=[i for i,x in enumerate(condition) if x==0]
    all_len = len(condition)-len(index0)
    index_all = [index0,index1,index2,index3,index4,index5]
    def cal_condp(patial_feature, ob):
        if ob==0:
            return [1,1,1,1,1]                                    # 如果ob到的结果是这个用户没有评测，则返回五个均1值
        patial_feature1= patial_feature[index1]
        patial_feature2= patial_feature[index2]
        patial_feature3= patial_feature[index3]
        patial_feature4= patial_feature[index4]
        patial_feature5= patial_feature[index5]
        num=0
        zero_num=0
        for i in patial_feature1:
            if i==ob:
                num=num+1
            if i==0:
                zero_num = zero_num+1
        condp1=(num+0.01)/(len(index1)-zero_num+0.05)
        num=0
        zero_num=0
        for i in patial_feature2:
            if i==ob:
                num=num+1
            if i==0:
                zero_num = zero_num+1
        condp2=(num+0.01)/(len(index2)-zero_num+0.05)
        num=0
        zero_num=0
        for i in patial_feature3:
            if i==ob:
                num=num+1
            if i==0:
                zero_num = zero_num+1
        condp3=(num+0.01)/(len(index3)-zero_num+0.05)
        num=0
        zero_num=0
        for i in patial_feature4:
            if i==ob:
                num=num+1
            if i==0:
                zero_num = zero_num+1
        condp4=(num+0.01)/(len(index4)-zero_num+0.05)
        num=0
        zero_num=0
        for i in patial_feature5:
            if i==ob:
                num=num+1
            if i==0:
                zero_num = zero_num+1
        condp5=(num+0.01)/(len(index5)-zero_num+0.05)
        return [condp1,condp2,condp3,condp4,condp5]
    condp_all = np.array([cal_condp(train_dfT[i],observe[i]) for i in range(len(observe))]) # 唯一区别，仅在这里使用转置训练数据
    condp1 = sum([math.log(i, 2) for i in condp_all[:,0]])+math.log((len(index1)+0.01)/(all_len+0.05), 2)
    condp2 = sum([math.log(i, 2) for i in condp_all[:,1]])+math.log((len(index2)+0.01)/(all_len+0.05), 2)
    condp3 = sum([math.log(i, 2) for i in condp_all[:,2]])+math.log((len(index3)+0.01)/(all_len+0.05), 2)
    condp4 = sum([math.log(i, 2) for i in condp_all[:,3]])+math.log((len(index4)+0.01)/(all_len+0.05), 2)
    condp5 = sum([math.log(i, 2) for i in condp_all[:,4]])+math.log((len(index5)+0.01)/(all_len+0.05), 2)
    all_condp = np.array([condp1,condp2,condp3,condp4,condp5])
    max_index = np.argmax(all_condp)
    all_condp = all_condp-all_condp[max_index]
    reli = 1/sum([2**i for i in all_condp])
    reli2 = (2**all_condp[3]+2**all_condp[4])/sum([2**i for i in all_condp])
    #condp1 = np.prod(condp_all[:,0])*(len(index1)+0.01)/(all_len+0.05)
    #condp2 = np.prod(condp_all[:,1])*(len(index2)+0.01)/(all_len+0.05)
    #condp3 = np.prod(condp_all[:,2])*(len(index3)+0.01)/(all_len+0.05)
    #condp4 = np.prod(condp_all[:,3])*(len(index4)+0.01)/(all_len+0.05)
    #condp5 = np.prod(condp_all[:,4])*(len(index5)+0.01)/(all_len+0.05)
    return [max_index+1, reli, reli2], [condp1,condp2,condp3,condp4,condp5]

In [47]:
def NBC_HBCF(condition, observe):
    num1=len([i for i,x in enumerate(observe) if x!=0])
    num2=len([i for i,x in enumerate(condition) if x!=0])
    _, a = NBC_UBCF(condition, observe)
    _, b = NBC_IBCF(observe, condition)
    all_condp = np.array(a)/(1+num1)+np.array(b)/(1+num2)
    return_out = list(all_condp)
    max_index = np.argmax(all_condp)
    all_condp = all_condp-all_condp[max_index]
    reli = 1/sum([2**i for i in all_condp])
    reli2 = (2**all_condp[3]+2**all_condp[4])/sum([2**i for i in all_condp])
    return [max_index+1, reli, reli2], return_out

In [115]:
train_df=np.array([[5,3,0,2,3,4,3,0],[1,2,0,2,4,4,5,0],[2,2,3,0,1,3,2,0],[1,5,5,4,4,5,2,0]])
train_dfT=np.transpose(train_df)
condition = [1,2,2,5,0,4,3,5]
observe = [1,1,3,5]                # 初始训练数据

In [116]:
a = NBC_UBCF(condition, observe)
print(a)
b = NBC_IBCF(observe, condition)   # 调用是观测与条件互换
print(b)
c = NBC_HBCF(condition, observe)
print(c)

([2, 0.5060677188251034], [-23.002038552447054, -16.28861581651964, -29.66025003519885, -16.343827069695262, -24.275092403992957])
([1, 0.9944569745412293], [-16.4262969544607, -24.91527476198352, -33.56043891037373, -24.91527476198352, -37.95275633315249])
([2, 0.3337842416936521], [-6.653694829796999, -6.372132508551868, -10.127104870836487, -6.383174759186993, -9.599113022442653])


In [6]:
zuoxiajiao = pd.read_csv('zuoxiajiao_df.csv')
zuoshangjiao = pd.read_csv('zuoshangjiao_df.csv')
youxiajiao = pd.read_csv('youxiajiao_df.csv')
youshangjiao = pd.read_csv('youshangjiao_df.csv')
train_df = zuoxiajiao.values
test_df = youshangjiao.values
train_df = train_df[:,1:]
train_dfT = np.transpose(train_df)
test_users = test_df[:,0]
test_items = list(youshangjiao.columns)[1:]

In [49]:
def cal_recommend(U, N):
    U_recommend = []
    U_recommend = list(U[:,2].argsort()[::-1][0:N])
    print(U_recommend)
    return U_recommend

In [7]:
def cal_recommend(U, N):
    U_recommend = []
    index_5 = np.where(U[:,0]==5)[0]
    index_4 = np.where(U[:,0]==4)[0]
    index_3 = np.where(U[:,0]==3)[0]
    index_2 = np.where(U[:,0]==2)[0]
    if (N<=len(index_5)):
        top_N_idx=U[:,1][index_5].argsort()[::-1][0:N]
        U_recommend = np.array(index_5)[top_N_idx]
    if (N>len(index_5) and N<=(len(index_5)+len(index_4))):
        top_N_idx=U[:,1][index_5].argsort()[::-1]
        U_recommend = np.array(index_5)[top_N_idx]
        top_N_idx=U[:,1][index_4].argsort()[::-1][0:(N-len(index_5))]
        U_recommend = list(U_recommend)+list(np.array(index_4)[top_N_idx])
    if (N<=(len(index_5)+len(index_4)+len(index_3)) and N>(len(index_5)+len(index_4))):
        top_N_idx=U[:,1][index_5].argsort()[::-1]
        U_recommend = np.array(index_5)[top_N_idx]
        top_N_idx=U[:,1][index_4].argsort()[::-1]
        U_recommend = list(U_recommend)+list(np.array(index_4)[top_N_idx])
        top_N_idx=U[:,1][index_3].argsort()[::-1][0:(N-len(index_5)-len(index_4))]
        U_recommend = list(U_recommend)+list(np.array(index_3)[top_N_idx])
    if (N<=(len(index_5)+len(index_4)+len(index_3)+len(index_2)) and N>(len(index_5)+len(index_4)+len(index_3))):
        top_N_idx=U[:,1][index_5].argsort()[::-1]
        U_recommend = np.array(index_5)[top_N_idx]
        top_N_idx=U[:,1][index_4].argsort()[::-1]
        U_recommend = list(U_recommend)+list(np.array(index_4)[top_N_idx])
        top_N_idx=U[:,1][index_3].argsort()[::-1]
        U_recommend = list(U_recommend)+list(np.array(index_3)[top_N_idx])
        top_N_idx=U[:,1][index_2].argsort()[::-1][0:(N-len(index_5)-len(index_4)-len(index_3))]
        U_recommend = list(U_recommend)+list(np.array(index_2)[top_N_idx])
    return U_recommend

In [51]:
def cal_4_index(U, T):
    N=2
    U_MAE=[]
    U_prec=[]
    U_recall=[]
    U_DCG=[]
    recommend_N = cal_recommend(U, 10) 
    trans_index = [sum(T>=2)+1,sum(T>=3)+1,sum(T>=4)+1,sum(T>=5)+1,1]
    while N<=10:
        U_MAE.append(metrics.mean_absolute_error(T, U[:,0]))
        U_prec.append(sum(T[recommend_N[:N]]>=4)/min(N,len(T)))
        U_recall.append(sum(T[recommend_N[:N]]>=4)/sum(T>=4))
        true_rank = [trans_index[T[i]-1] for i in recommend_N[:N]]
        U_DCG.append(getNDCG(true_rank, list(range(1, min(len(T),N)+1))))
        N = N+1
    return [U_MAE, U_prec, U_recall, U_DCG]

In [52]:
U_pred_label = []
I_pred_label = []
H_pred_label = []
true_label = []
U_index_all = []
I_index_all = []
H_index_all = []
for i in range(400,600):
    sub_U_pred = []
    sub_I_pred = []
    sub_H_pred = []
    sub_true = []
    condition = zuoshangjiao.loc[[i]]
    condition = condition.values
    condition = condition[0][1:]
    for j in test_items:
        observe = youxiajiao[j]
        observe = observe.values
        sub_true_label = youshangjiao[j].loc[[i]].values[0]
        if (sub_true_label==0):
            continue
        pred_label_U, _ = NBC_UBCF(condition, observe)
        pred_label_I, _ = NBC_IBCF(observe, condition)
        pred_label_H, _ = NBC_HBCF(condition, observe)
        sub_U_pred.append(pred_label_U)
        sub_I_pred.append(pred_label_I)
        sub_H_pred.append(pred_label_H)
        sub_true.append(sub_true_label)
        U_pred_label.append(pred_label_U[0])
        I_pred_label.append(pred_label_I[0])
        H_pred_label.append(pred_label_H[0])
        true_label.append(sub_true_label)
    U_index_all.append(cal_4_index(np.array(sub_U_pred), np.array(sub_true)))
    I_index_all.append(cal_4_index(np.array(sub_I_pred), np.array(sub_true)))
    H_index_all.append(cal_4_index(np.array(sub_H_pred), np.array(sub_true)))

[6, 1, 7, 5, 4, 3, 2, 0]
[3, 1, 2, 7, 4, 0, 6, 5]
[2, 1, 0, 4, 7, 6, 5, 3]
[7, 6, 5, 4, 3, 2, 1, 0]
[1, 7, 0, 6, 4, 2, 5, 3]
[7, 1, 0, 4, 2, 5, 3, 6]
[71, 24, 39, 38, 37, 36, 32, 31, 29, 28]
[71, 62, 31, 29, 49, 51, 22, 24, 7, 9]
[9, 31, 0, 49, 25, 71, 28, 7, 4, 14]
[4, 5, 3, 1, 2, 0]
[5, 2, 0, 3, 1, 4]
[5, 2, 0, 3, 1, 4]
[0, 142, 145, 43, 42, 76, 165, 37, 176, 177]
[252, 187, 85, 86, 93, 97, 100, 104, 105, 106]
[24, 74, 8, 21, 250, 208, 73, 145, 80, 197]
[0, 9, 20, 19, 3, 8, 18, 13, 7, 2]
[20, 17, 9, 19, 7, 0, 5, 8, 6, 14]
[20, 9, 19, 3, 0, 8, 4, 15, 7, 5]
[11, 12, 1, 2, 3, 5, 6, 7, 8, 10]
[19, 17, 4, 2, 7, 13, 11, 1, 16, 8]
[17, 19, 2, 11, 7, 16, 6, 8, 5, 4]
[32, 31, 1, 2, 3, 4, 5, 6, 7, 8]
[32, 23, 6, 8, 9, 31, 19, 22, 0, 26]
[19, 32, 7, 8, 9, 6, 18, 25, 0, 3]
[18, 5, 16, 1, 14, 4, 3, 8, 13, 17]
[18, 1, 13, 4, 10, 14, 16, 17, 5, 12]
[18, 1, 5, 14, 16, 13, 7, 3, 4, 10]
[9, 3, 6, 10, 7, 2, 8, 5, 4, 1]
[8, 10, 0, 1, 5, 2, 6, 4, 9, 3]
[9, 8, 0, 3, 1, 10, 5, 2, 6, 4]
[13, 16, 2, 6, 10, 1

[4, 5, 1, 2, 0, 3]
[5, 4, 2, 0, 3, 1]
[4, 5, 2, 0, 3, 1]
[4, 1, 3, 0, 2]
[1, 0, 4, 2, 3]
[1, 4, 0, 3, 2]
[2, 4, 7, 3, 9, 8, 0, 1, 6, 5]
[2, 7, 0, 9, 3, 5, 8, 4, 1, 6]
[2, 9, 8, 0, 3, 4, 1, 5, 7, 6]
[3, 2, 1, 0]
[1, 3, 2, 0]
[1, 0, 2, 3]
[7, 4, 3, 2, 0, 9, 6, 8, 5, 1]
[0, 2, 6, 10, 7, 5, 3, 1, 9, 8]
[2, 7, 0, 3, 6, 5, 10, 4, 9, 8]
[5, 4, 7, 3, 6, 0, 2, 1]
[2, 5, 1, 7, 4, 6, 3, 0]
[5, 2, 1, 7, 4, 3, 6, 0]
[4, 2, 1, 3, 0]
[0, 2, 1, 4, 3]
[0, 2, 1, 4, 3]
[26, 14, 1, 2, 4, 7, 8, 9, 10, 12]
[25, 26, 7, 2, 21, 8, 17, 14, 10, 11]
[7, 25, 8, 17, 10, 24, 26, 5, 2, 23]
[34, 13, 1, 2, 3, 4, 5, 6, 9, 11]
[34, 4, 11, 32, 26, 0, 14, 19, 2, 6]
[4, 0, 12, 24, 34, 6, 11, 5, 1, 31]
[6, 0, 7, 4, 5, 1, 2, 3]
[1, 5, 6, 7, 0, 2, 3, 4]
[0, 1, 6, 7, 5, 3, 4, 2]
[38, 9, 16, 15, 14, 13, 12, 11, 10, 8]
[38, 21, 13, 27, 11, 3, 14, 37, 7, 19]
[12, 13, 11, 14, 2, 38, 7, 0, 20, 3]
[0, 37, 22, 21, 20, 32, 19, 14, 13, 27]
[19, 20, 32, 42, 0, 45, 1, 37, 41, 2]
[32, 20, 19, 0, 22, 2, 21, 37, 14, 42]
[0, 3, 1, 5, 4, 2]
[1

i:\机器人小组专用\深度学习处理图像\python\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in long_scalars
  if sys.path[0] == '':
i:\机器人小组专用\深度学习处理图像\python\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in long_scalars
  if sys.path[0] == '':
i:\机器人小组专用\深度学习处理图像\python\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in long_scalars
  if sys.path[0] == '':
i:\机器人小组专用\深度学习处理图像\python\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in long_scalars
  if sys.path[0] == '':
i:\机器人小组专用\深度学习处理图像\python\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in long_scalars
  if sys.path[0] == '':
i:\机器人小组专用\深度学习处理图像\python\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in long_scalars
  if sys.path[0] == '':
i:\机器人小组专用\深度学习处理图像\python\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarn

[0, 1, 3, 2]
[2, 0, 1, 3]
[0, 1, 2, 3]
[0, 49, 30, 26, 32, 65, 35, 24, 39, 21]
[31, 4, 52, 28, 13, 55, 2, 57, 29, 34]
[55, 61, 9, 13, 20, 8, 63, 24, 28, 39]
[10, 8, 5, 3, 2, 0, 1, 9, 7, 6]
[2, 1, 10, 0, 8, 6, 7, 5, 4, 3]
[2, 10, 0, 8, 1, 3, 5, 6, 7, 9]
[3, 1, 0, 5, 8, 4, 9, 10, 7, 6]
[1, 10, 0, 3, 7, 4, 8, 2, 9, 6]
[1, 3, 0, 10, 4, 7, 8, 9, 2, 6]
[11, 9, 2, 1, 6, 5, 4, 8, 0, 7]
[0, 11, 4, 7, 9, 1, 5, 6, 2, 10]
[11, 0, 1, 9, 4, 6, 5, 2, 7, 8]
[0, 41, 9, 8, 37, 38, 39, 40, 83, 15]
[42, 31, 23, 24, 25, 26, 28, 29, 30, 32]
[2, 68, 49, 23, 74, 45, 29, 28, 7, 26]
[6, 21, 8, 28, 14, 25, 17, 1, 11, 4]
[18, 13, 8, 3, 1, 24, 2, 21, 28, 20]
[8, 6, 1, 14, 20, 25, 18, 2, 3, 17]
[89, 33, 25, 26, 27, 28, 29, 30, 31, 32]
[89, 55, 65, 23, 62, 25, 60, 59, 26, 27]
[76, 11, 27, 30, 33, 28, 10, 29, 88, 62]
[7, 6, 5, 4, 3, 2, 1, 0]
[3, 1, 7, 2, 6, 4, 5, 0]
[3, 1, 5, 7, 2, 4, 6, 0]
[65, 66, 51, 52, 54, 55, 57, 61, 64, 67]
[131, 83, 44, 45, 46, 47, 48, 50, 51, 52]
[43, 48, 55, 45, 47, 90, 85, 50, 44, 52]
[0, 

In [41]:
U_index_all = np.array(U_index_all)
I_index_all = np.array(I_index_all)
H_index_all = np.array(H_index_all)
U_MAE = sum(U_index_all[:, 0])/len(U_index_all[:, 0])
U_prec = sum(U_index_all[:, 1])/len(U_index_all[:, 1])
U_recall = sum(U_index_all[:, 2])/len(U_index_all[:, 2])
U_DCG = sum(U_index_all[:, 3])/len(U_index_all[:, 3])
I_MAE = sum(I_index_all[:, 0])/len(I_index_all[:, 0])
I_prec = sum(I_index_all[:, 1])/len(I_index_all[:, 1])
I_recall = sum(I_index_all[:, 2])/len(I_index_all[:, 2])
I_DCG = sum(I_index_all[:, 3])/len(I_index_all[:, 3])
H_MAE = sum(H_index_all[:, 0])/len(H_index_all[:, 0])
H_prec = sum(H_index_all[:, 1])/len(H_index_all[:, 1])
H_recall = sum(H_index_all[:, 2])/len(H_index_all[:, 2])
H_DCG = sum(H_index_all[:, 3])/len(H_index_all[:, 3])
print(U_MAE, I_MAE, H_MAE)
print(U_prec, I_prec, H_prec)
print(U_recall, I_recall, H_recall)
print(U_DCG, I_DCG, H_DCG)

1.3723672408707888 0.8014246033250276 0.8577649408714914
0.4925 0.565625 0.595
nan nan nan
0.4235953688621521 0.5325031883269549 0.5690156688168645


In [53]:
data_all_4 = [U_index_all,I_index_all,H_index_all]
np.save('data_all_4', data_all_4)

In [42]:
U_recall_data = []
I_recall_data = []
H_recall_data = []
for i in U_index_all[:, 2]:
    if np.isnan(i):
        continue
    U_recall_data.append(i)
for i in I_index_all[:, 2]:
    if np.isnan(i):
        continue
    I_recall_data.append(i)
for i in H_index_all[:, 2]:
    if np.isnan(i):
        continue
    H_recall_data.append(i)
print(sum(U_recall_data)/len(U_recall_data), sum(I_recall_data)/len(I_recall_data), sum(H_recall_data)/len(H_recall_data))

0.21835082406027137 0.3153763031387148 0.3087283507073314


In [56]:
label_data = [U_pred_label, I_pred_label, H_pred_label, true_label]
np.save('label_all_4', label_data)

In [43]:
save_data=[[U_MAE,I_MAE, H_MAE],[U_prec, I_prec, H_prec],[sum(U_recall_data)/len(U_recall_data), sum(I_recall_data)/len(I_recall_data), sum(H_recall_data)/len(H_recall_data)],[U_DCG, I_DCG, H_DCG]]
np.save('data600_800',save_data)

In [10]:
def getDCG(scores):
    return np.sum(
        np.divide(np.power(2, scores) - 1, np.log(np.arange(scores.shape[0], dtype=np.float32) + 2)),
        dtype=np.float32)
def getNDCG(rank_list, pos_items):
    relevance = np.ones_like(pos_items)
    it2rel = {it: r for it, r in zip(pos_items, relevance)}
    rank_scores = np.asarray([it2rel.get(it, 0.0) for it in rank_list], dtype=np.float32)

    idcg = getDCG(relevance)

    dcg = getDCG(rank_scores)

    if dcg == 0.0:
        return 0.0
    ndcg = dcg / idcg
    return ndcg

In [55]:
U_pred_label = []
I_pred_label = []
H_pred_label = []
true_label = []
U_index_all = []
I_index_all = []
H_index_all = []
num = 8
for j in test_items[318:424]:
    observe = youxiajiao[j]
    observe = observe.values
    sub_U_pred = []
    sub_I_pred = []
    sub_H_pred = []
    sub_true = []
    for i in range(0,1200):
        condition = zuoshangjiao.loc[[i]]
        condition = condition.values
        condition = condition[0][1:]
        sub_true_label = youshangjiao[j].loc[[i]].values[0]
        if (sub_true_label==0):
            continue
        
        pred_label_U, _ = NBC_UBCF(condition, observe)
        pred_label_I, _ = NBC_IBCF(observe, condition)
        pred_label_H, _ = NBC_HBCF(condition, observe)
        sub_U_pred.append(pred_label_U)
        sub_I_pred.append(pred_label_I)
        sub_H_pred.append(pred_label_H)
        sub_true.append(sub_true_label)
        U_pred_label.append(pred_label_U)
        I_pred_label.append(pred_label_I)
        H_pred_label.append(pred_label_H)
        true_label.append(sub_true_label)
    if (len(sub_true)==0):
        continue
    U_index_all.append(metrics.mean_absolute_error(sub_true, np.array(sub_U_pred)[:,0]))
    I_index_all.append(metrics.mean_absolute_error(sub_true, np.array(sub_I_pred)[:,0]))
    H_index_all.append(metrics.mean_absolute_error(sub_true, np.array(sub_H_pred)[:,0]))

In [57]:
print(sum(U_index_all)/len(U_index_all), sum(I_index_all)/len(I_index_all), sum(H_index_all)/len(H_index_all))

0.8685433803022219 1.2143150046576094 0.8300449467274742
